In [26]:
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

Importar los datos

In [27]:
admin_src_df = pd.read_csv('data/admission_source_id.csv', 
                           na_values=[" Not Available","Not Available","None","Unknown/Invalid","Not Mapped"," Not Mapped"])
admin_type_df = pd.read_csv('data/admission_type_id.csv',
                           na_values = ["Not Available","None","Not Mapped"])
diabetic_df = pd.read_csv('data/diabetic_data.csv',
                          na_values=["Not Available","None","Unknown/Invalid","Not Mapped","?"])
dschrge_disp_df = pd.read_csv('data/discharge_disposition_id.csv',
                             na_values=["Not Available","None","Unknown/Invalid","Not Mapped"])
#id_mapping_df = pd.read_csv('data/IDs_mapping.csv')    #Es la colección de todas las demás tabals de referencia

C:\Users\bdih7\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [28]:
diabetic_df

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),NaN,6,25,1,1,...,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),NaN,1,1,7,3,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),NaN,1,1,7,2,...,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),NaN,1,1,7,2,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),NaN,1,1,7,1,...,No,Steady,No,No,No,No,No,Ch,Yes,NO
5,35754,82637451,Caucasian,Male,[50-60),NaN,2,1,2,3,...,No,Steady,No,No,No,No,No,No,Yes,>30
6,55842,84259809,Caucasian,Male,[60-70),NaN,3,1,2,4,...,No,Steady,No,No,No,No,No,Ch,Yes,NO
7,63768,114882984,Caucasian,Male,[70-80),NaN,1,1,7,5,...,No,No,No,No,No,No,No,No,Yes,>30
8,12522,48330783,Caucasian,Female,[80-90),NaN,2,1,4,13,...,No,Steady,No,No,No,No,No,Ch,Yes,NO
9,15738,63555939,Caucasian,Female,[90-100),NaN,3,3,4,12,...,No,Steady,No,No,No,No,No,Ch,Yes,NO


Revisemos qué hay en cada uno de los dataframes

In [29]:
admin_src_df
#admin_type_df
#dschrge_disp_df
#diabetic_df.head()

,admission_source_id,description
0,1,Physician Referral
1,2,Clinic Referral
2,3,HMO Referral
3,4,Transfer from a hospital
4,5,Transfer from a Skilled Nursing Facility (SNF)
5,6,Transfer from another health care facility
6,7,Emergency Room
7,8,Court/Law Enforcement
8,9,NaN
9,10,Transfer from critial access hospital


Asignemos los índices originales a los ficheros de las tabals de referencia para desnormalizar la tabla _diabetic_ _ _df_

In [30]:
admin_src_df.set_index("admission_source_id", inplace = True)
admin_type_df.set_index("admission_type_id", inplace = True)
dschrge_disp_df.set_index("discharge_disposition_id", inplace = True)

Cambiemos el nombre de las descripciones de las tablas de referencia

In [31]:
admin_src_df.rename(columns={"description":"admin_src_desc"}, inplace = True)
admin_type_df.rename(columns={"description":"admin_type_desc"}, inplace = True)
dschrge_disp_df.rename(columns={"description":"dschrge_disp_desc"}, inplace = True)

Ahora sí ejecutemos los joins. Verifiquemos que si las relaciones son en efecto uno a uno

In [32]:
print(diabetic_df.join(admin_src_df,how="left", on = "admission_source_id").shape)
print(diabetic_df.join(admin_type_df,how="left", on = "admission_type_id").shape)
print(diabetic_df.join(dschrge_disp_df,how="left", on = "discharge_disposition_id").shape)
print(diabetic_df.shape)

(101766, 51)
(101766, 51)
(101766, 51)
(101766, 50)


In [33]:
aug_diabetic_df = diabetic_df.join(admin_src_df,how="left", on="admission_source_id")
aug_diabetic_df = aug_diabetic_df.join(admin_type_df,how="left", on = "admission_type_id")
aug_diabetic_df = aug_diabetic_df.join(dschrge_disp_df,how="left", on = "discharge_disposition_id")

Quitemos los id's que ya denormalizamos

In [34]:
aug_diabetic_df.drop(labels = ["admission_type_id","discharge_disposition_id","admission_source_id"], 
                     axis = 1, 
                     inplace = True)

In [35]:
aug_diabetic_df

,encounter_id,patient_nbr,race,gender,age,weight,time_in_hospital,payer_code,medical_specialty,num_lab_procedures,...,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted,admin_src_desc,admin_type_desc,dschrge_disp_desc
0,2278392,8222157,Caucasian,Female,[0-10),NaN,1,NaN,Pediatrics-Endocrinology,41,...,No,No,No,No,No,No,NO,Physician Referral,NaN,NaN
1,149190,55629189,Caucasian,Female,[10-20),NaN,3,NaN,NaN,59,...,No,No,No,No,Ch,Yes,>30,Emergency Room,Emergency,Discharged to home
2,64410,86047875,AfricanAmerican,Female,[20-30),NaN,2,NaN,NaN,11,...,No,No,No,No,No,Yes,NO,Emergency Room,Emergency,Discharged to home
3,500364,82442376,Caucasian,Male,[30-40),NaN,2,NaN,NaN,44,...,No,No,No,No,Ch,Yes,NO,Emergency Room,Emergency,Discharged to home
4,16680,42519267,Caucasian,Male,[40-50),NaN,1,NaN,NaN,51,...,No,No,No,No,Ch,Yes,NO,Emergency Room,Emergency,Discharged to home
5,35754,82637451,Caucasian,Male,[50-60),NaN,3,NaN,NaN,31,...,No,No,No,No,No,Yes,>30,Clinic Referral,Urgent,Discharged to home
6,55842,84259809,Caucasian,Male,[60-70),NaN,4,NaN,NaN,70,...,No,No,No,No,Ch,Yes,NO,Clinic Referral,Elective,Discharged to home
7,63768,114882984,Caucasian,Male,[70-80),NaN,5,NaN,NaN,73,...,No,No,No,No,No,Yes,>30,Emergency Room,Emergency,Discharged to home
8,12522,48330783,Caucasian,Female,[80-90),NaN,13,NaN,NaN,68,...,No,No,No,No,Ch,Yes,NO,Transfer from a hospital,Urgent,Discharged to home
9,15738,63555939,Caucasian,Female,[90-100),NaN,12,NaN,InternalMedicine,33,...,No,No,No,No,Ch,Yes,NO,Transfer from a hospital,Elective,Discharged/transferred to SNF


Ahora entendamos la información que tenemos en el _dataset_

In [36]:
aug_diabetic_df.set_index(["patient_nbr","encounter_id"], inplace = True)

## Verifiquemos valores nulos de variables importantes

Antes de revisar valores nulos mal codificados, verifiquemos el total de valores nulos ya identificados

In [37]:
aug_diabetic_df.isna().sum()/aug_diabetic_df.shape[0]

race                        0.022336
gender                      0.000029
age                         0.000000
weight                      0.968585
time_in_hospital            0.000000
payer_code                  0.395574
medical_specialty           0.490822
num_lab_procedures          0.000000
num_procedures              0.000000
num_medications             0.000000
number_outpatient           0.000000
number_emergency            0.000000
number_inpatient            0.000000
diag_1                      0.000206
diag_2                      0.003518
diag_3                      0.013983
number_diagnoses            0.000000
max_glu_serum               0.947468
A1Cresult                   0.832773
metformin                   0.000000
repaglinide                 0.000000
nateglinide                 0.000000
chlorpropamide              0.000000
glimepiride                 0.000000
acetohexamide               0.000000
glipizide                   0.000000
glyburide                   0.000000
t

Hasta el momento tenemos algunos _features_ con hasta 94 y 96% de valores nulos. Podríamos eliminarlos, pero como son _features_ importantes para este tipo de problema, el peso y el nivel máximo de glucosa, codifiquemos explícitamente el NA

In [38]:
aug_diabetic_df.weight[~aug_diabetic_df.weight.isna()]

patient_nbr  encounter_id
94466574     8042082          [75-100)
96198543     8057670           [50-75)
79874631     8235036            [0-25)
90421380     8274126          [75-100)
55628172     8421486          [75-100)
80041266     8576406           [50-75)
93126573     8620524          [75-100)
10648701     8753532          [75-100)
103586670    8866632         [100-125)
96007464     9075132          [75-100)
67852377     9076452          [75-100)
42912702     9089646           [50-75)
81133947     9198750          [75-100)
78871527     9327402          [75-100)
42516765     9813246          [75-100)
50444955     10320138         [75-100)
25739460     11190414         [75-100)
100526274    11194338         [75-100)
54126873     16173522          [50-75)
105841233    16203084         [75-100)
98466363     16319028          [50-75)
56032524     16396500         [75-100)
107804349    16623912          [50-75)
33368364     16732086           [0-25)
106625952    16789716         [75-100)

In [39]:
aug_diabetic_df.max_glu_serum[~aug_diabetic_df.max_glu_serum.isna()]

patient_nbr  encounter_id
44279478     2265486         >300
53848278     2292606         >300
77475465     2309268         Norm
95586993     2311404         Norm
69318126     2430276         Norm
80499960     2521974         >200
48452823     2548314         Norm
89193870     2595612         >300
80156115     2660244         Norm
105589584    2791716         >200
86209290     2926554         Norm
95564241     3086520         Norm
103892157    3092106         >300
818874       3119718         Norm
108453753    3120912         >200
63285453     3148422         Norm
71235666     3358950         Norm
25831539     3616560         Norm
68029074     3686592         Norm
100533195    3783912         >300
34568172     3977208         >200
70269183     4066842         >300
76959585     4084524         >300
86797872     4211154         >200
104789817    4296672         >300
114960726    4804968         >300
73286937     5294886         >200
87847776     5319174         >200
83268891     5593122  

cambiar nombres

In [41]:
aug_diabetic_df.columns = aug_diabetic_df.columns.str.lower().str.strip(' ').str.replace(pat="-",repl="_")

In [47]:
aug_diabetic_df.rename(columns={"diabetesmed":"diabetes_med"},inplace=True)

In [46]:
aug_diabetic_df.columns

Index(['race', 'gender', 'age', 'weight', 'time_in_hospital', 'payer_code',
       'medical_specialty', 'num_lab_procedures', 'num_procedures',
       'num_medications', 'number_outpatient', 'number_emergency',
       'number_inpatient', 'diag_1', 'diag_2', 'diag_3', 'number_diagnoses',
       'max_glu_serum', 'a1cresult', 'metformin', 'repaglinide', 'nateglinide',
       'chlorpropamide', 'glimepiride', 'acetohexamide', 'glipizide',
       'glyburide', 'tolbutamide', 'pioglitazone', 'rosiglitazone', 'acarbose',
       'miglitol', 'troglitazone', 'tolazamide', 'examide', 'citoglipton',
       'insulin', 'glyburide_metformin', 'glipizide_metformin',
       'glimepiride_pioglitazone', 'metformin_rosiglitazone',
       'metformin_pioglitazone', 'change', 'diabetesmed', 'readmitted',
       'admin_src_desc', 'admin_type_desc', 'dschrge_disp_desc'],
      dtype='object')